# Demo

## Load Environment Vars

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

## Data Models

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Subcategory(BaseModel):
    """A project subcategory."""
    
    id: int = Field(description="The subcategory id.")
    name: str = Field(description="The subcategory name.")

class ReprovationReason(BaseModel):
    """A project reprovation reason."""
    
    id: int = Field(description="The reprovation reason id.")
    name: str = Field(description="The reprovation reason name.")
    description: str = Field(description="The reprovation reason description.")

class ProjectInput(BaseModel):
    """The project data defined by the client (may need changes)."""
    
    subcategory: int = Field(description="The subcategory of the project.")
    title: str = Field(description="The title of the project.")
    description: str = Field(description="The description of the project.")
    abilities: list[str] | None = Field(description="The abilities desired for the project.")
    
class ProjectPrediction(BaseModel):
    """The predicted project data based on the input. Some fields might be left undefined depending on the project being approved or not."""
    
    approve: bool = Field(description="Whether the project should be approved or not.")
    subcategory: int | None = Field(description="The subcategory of the project (when approved).")
    title: str | None = Field(description="The title of the project (when approved).", min_length=25, max_length=75)
    description: str | None = Field(description="The description of the project (when approved).", min_length=50, max_length=2000)
    reprovation_reason: int | None = Field(description="The reason for the project to be reproved (when reproved).")
    reprovation_comment: str | None = Field(description="The comment for the reprovation (when reproved).")
    
class ProjectInfo(BaseModel):
    """Additional information about the project."""
    
    reasoning: str = Field(description="The reasoning behind the prediction.")
    confidence: float = Field(description="The confidence of the prediction.", ge=0, le=1)
    bot_generated: float = Field(description="How likely is for the project to be bot generated.", ge=0, le=1)
    
    
class ProjectOutput(BaseModel):
    """The predicted project data based on the input, along with additional information."""
    
    prediction: ProjectPrediction = Field(description="The predicted project data based on the input.")
    info: ProjectInfo = Field(description="Additional information about the project.")

## Load Examples

In [3]:
def get_subcategories():
    return [
        Subcategory(id=0, name="Outra categoria"),
        Subcategory(id=1, name="Design gráfico"),
        Subcategory(id=2, name="Desenvolvimento Web"),
        Subcategory(id=3, name="Redação"),
        Subcategory(id=4, name="Edição de Vídeo"),
        Subcategory(id=5, name="Tradução"),
        Subcategory(id=6, name="Assistente Virtual"),
        Subcategory(id=7, name="Animação"),
        Subcategory(id=8, name="Marketing Digital"),
        Subcategory(id=9, name="Ilustração"),
    ]

def get_reprovation_reasons():
    return [
        ReprovationReason(id=1, name="Outro Motivo", description=""),
        ReprovationReason(id=2, name="Projeto Duplicado", description="Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo"),
        ReprovationReason(id=3, name="Proposta para Trabalhar", description="Se o seu objetivo é encontrar trabalhos, por favor, altere o seu perfil para o de Freelancer"),
        ReprovationReason(id=4, name="Conteúdo Não Permitido", description="Há conteúdo não permitido na descrição do projeto (informações de contato, orçamento, solicitação de testes não remunerados, etc)"),
        ReprovationReason(id=5, name="Projeto Confuso", description="Não está claro o que é pedido no projeto"),
        ReprovationReason(id=6, name="Poucos Detalhes", description="Os freelancers precisam saber detalhes para que possam enviar as suas propostas. Seja o mais específico possível"),
        ReprovationReason(id=7, name="Pagamento Comissionado", description="Projetos com pagamentos parciais ou totais via comissão não são permitidos"),
        ReprovationReason(id=8, name="Requer trabalho não pago", description="Todo trabalho deve ser pago"),
        ReprovationReason(id=9, name="Troca de trabalho", description="Permutas não são permitidas"),
        ReprovationReason(id=10, name="Vaga de emprego", description="Nosso site é somente para trabalhos temporários. Não podemos te ajudar a encontrar profissionais para ocupar cargos na sua empresa"),
        ReprovationReason(id=11, name="Trabalhos acadêmicos", description="O seu trabalho não pode ser feito por outra pessoa. Valorize a sua educação"),
        ReprovationReason(id=12, name="Ilegal", description="Está solicitando algo que é ilegal ou que será utilizado para fins não permitidos"),
        ReprovationReason(id=13, name="Sem demanda", description="Está recrutando profissionais para parceria ou para demandas futuras"),
        ReprovationReason(id=14, name="Não Permitido", description="Esse tipo de trabalho não é permitido"),
    ]

def get_full_example(idx):
    project_id = idx + 1
    
    if idx % 5 == 0:
        input = ProjectInput(
            subcategory=1,
            title=f"PRECISO DE DESIGNER GRFICO - #{project_id}",
            description=f"Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321",
            abilities=["Criação de logotipo", "Ilustração"],
        )
        prediction = ProjectPrediction(
            approve=True,
            subcategory=1,
            title=f"Designer gráfico para criação de Logotipo - #{project_id}",
            description=f"Preciso de designer gráfico para criação de logotipo.",
        )
        info = ProjectInfo(
            reasoning="Alteração do título para ficar mais conciso e ajustes em sua capitalização. Correções ortográficas na descrição. Remoção de telefone de contato.",
            confidence=0.8,
            bot_generated=0.4,
        )
    elif idx % 5 == 1:
        input = ProjectInput(
            subcategory=0,
            title=f"ESCREVER ARTIGOS PARA BLOG - #{project_id}",
            description=f"Preciso de redaor para criação de conteúdo para meu blog https://meublog.com",
        )
        prediction = ProjectPrediction(
            approve=True,
            subcategory=3,
            title=f"Escrever artigos para blog - #{project_id}",
            description=f"Preciso de redator para criação de conteúdo para meu blog https://meublog.com",
        )
        info = ProjectInfo(
            reasoning="Ajustes na capitalização do título. Correção de erro ortográfico na descrição.",
            confidence=0.7,
            bot_generated=0.3,
        )
    elif idx % 5 == 2:
        input = ProjectInput(
            subcategory=0,
            title=f"TRADUÇÃO DE TEXTO - #{project_id}",
            description=f"Preciso de tradutor para traduzir texto de inglês para português. Primeiramente será feito um teste não pago para depois ser decidido o freelancer que fará o projeto.",
        )
        prediction = ProjectPrediction(
            approve=False,
            reprovation_reason=8,            
        )
        info = ProjectInfo(
            reasoning="Testes não pagos não são permitidos.",
            confidence=0.8,
            bot_generated=0.3,
        )
    elif idx % 5 == 3:
        input = ProjectInput(
            subcategory=8,
            title=f"Assistente virtual para tarefa pontual - #{project_id}",
            description=f"Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.",
        )
        prediction = ProjectPrediction(
            approve=True,
            subcategory=6,
            title=f"Assistente virtual para tarefa pontual - #{project_id}",
            description=f"Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.",
        )
        info = ProjectInfo(
            reasoning="Nenhuma alteração necessária.",
            confidence=0.9,
            bot_generated=0.2,
        )
    elif idx % 5 == 4:
        input = ProjectInput(
            subcategory=0,
            title=f"Crawler de Website - #{project_id}",
            description=f"Preciso de alguém que faça um crawler para coletar informações de um website. Deverá acessar a API e burlar o CAPTCHA, se necessário.",
        )
        prediction = ProjectPrediction(
            approve=False,
            reprovation_reason=12,
            reprovation_comment="Solicitação de algo não permitido (burlar CAPTCHA).",
        )
        info = ProjectInfo(
            reasoning="Solicitação de algo não permitido (burlar CAPTCHA).",
            confidence=0.8,
            bot_generated=0.3,
        )

    output = ProjectOutput(prediction=prediction, info=info)
    return project_id, input, output

# Examples of a pretend task of creating antonyms.
examples = [get_full_example(i) for i in range(100)]

In [4]:
examples[:2]

[(1,
  ProjectInput(subcategory=1, title='PRECISO DE DESIGNER GRFICO - #1', description='Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321', abilities=['Criação de logotipo', 'Ilustração']),
  ProjectOutput(prediction=ProjectPrediction(approve=True, subcategory=1, title='Designer gráfico para criação de Logotipo - #1', description='Preciso de designer gráfico para criação de logotipo.', reprovation_reason=None, reprovation_comment=None), info=ProjectInfo(reasoning='Alteração do título para ficar mais conciso e ajustes em sua capitalização. Correções ortográficas na descrição. Remoção de telefone de contato.', confidence=0.8, bot_generated=0.4))),
 (2,
  ProjectInput(subcategory=0, title='ESCREVER ARTIGOS PARA BLOG - #2', description='Preciso de redaor para criação de conteúdo para meu blog https://meublog.com', abilities=None),
  ProjectOutput(prediction=ProjectPrediction(approve=True, subcategory=3, title='Escrever artigos para blog - #2', description='Pre

## Create Store

In [5]:
from langchain_core.vectorstores import VectorStore
import typing

T = typing.TypeVar('T')

class MyVectorStores:
    def __init__(
            self,
            subcategories: VectorStore,
            reprovation_reasons: VectorStore,
            project_inputs: VectorStore,
            project_predictions: VectorStore):
        self.subcategories = subcategories
        self.reprovation_reasons = reprovation_reasons
        self.project_inputs = project_inputs
        self.project_predictions = project_predictions

class VectorStoreHandler(typing.Generic[T]):
    def __init__(self, store: VectorStore, json_parser: typing.Callable[[str], T]):
        self.store = store
        self.json_parser = json_parser
        
    def get_by_id(self, id: int):
        documents = self.store.get(ids=[str(id)])['documents']
        document = documents[0] if len(documents or []) else None
        data = self.json_parser(document) if document else None
        return data
    
    def similarity_search(
            self, 
            query: str, 
            k: int = 5):
        documents = self.store.similarity_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.page_content), documents))
    
    async def asimilarity_search(
            self, 
            query: str, 
            k: int = 5):
        documents = await self.store.asimilarity_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.page_content), documents))
    
    def max_marginal_relevance_search(
            self, 
            query: str, 
            k: int = 5):
        documents = self.store.max_marginal_relevance_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.page_content), documents))
    
    async def amax_marginal_relevance_search(
            self, 
            query: str, 
            k: int = 5):
        documents = await self.store.amax_marginal_relevance_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.page_content), documents))

class MyVectorStoreHandler:
    def __init__(self, stores: MyVectorStores):
        self.subcategories = VectorStoreHandler(store=stores.subcategories, json_parser=Subcategory.parse_raw)
        self.reprovation_reasons = VectorStoreHandler(store=stores.reprovation_reasons, json_parser=ReprovationReason.parse_raw)
        self.project_inputs = VectorStoreHandler(store=stores.project_inputs, json_parser=ProjectInput.parse_raw)
        self.project_predictions = VectorStoreHandler(store=stores.project_predictions, json_parser=ProjectPrediction.parse_raw)

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

data_dir = "./data/chroma_db"
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def create_store(collection_name):
    return Chroma(persist_directory=data_dir, collection_name=collection_name, embedding_function=embedding_function)

def create_stores():
    stores = MyVectorStores(
        subcategories=create_store("subcategories"),
        reprovation_reasons=create_store("reprovation_reasons"),
        project_inputs=create_store("project_inputs"),
        project_predictions=create_store("project_predictions"),
    )
    stores_handler = MyVectorStoreHandler(stores)
    return stores, stores_handler

stores, stores_handler = create_stores()

c:\Users\Asus\anaconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Asus\anaconda3\envs\dev\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
[stores.subcategories.add_texts(ids=[str(subcategory.id)], texts=[subcategory.json()]) for subcategory in get_subcategories()]
[stores.reprovation_reasons.add_texts(ids=[str(reason.id)], texts=[reason.json()]) for reason in get_reprovation_reasons()]
[stores.project_inputs.add_texts(ids=[str(project_id)], texts=[input.json()]) for project_id, input, _ in examples]
[stores.project_predictions.add_texts(ids=[str(project_id)], texts=[output.prediction.json()]) for project_id, _, output in examples]

stores, stores_handler = create_stores()

In [8]:
from pprint import pprint

In [9]:
pprint(stores_handler.subcategories.similarity_search("design", k=3))

[Subcategory(id=1, name='Design gráfico'),
 Subcategory(id=8, name='Marketing Digital'),
 Subcategory(id=0, name='Outra categoria')]


In [10]:
pprint(stores_handler.reprovation_reasons.similarity_search("duplicado", k=3))

[ReprovationReason(id=2, name='Projeto Duplicado', description='Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo'),
 ReprovationReason(id=5, name='Projeto Confuso', description='Não está claro o que é pedido no projeto'),
 ReprovationReason(id=4, name='Conteúdo Não Permitido', description='Há conteúdo não permitido na descrição do projeto (informações de contato, orçamento, solicitação de testes não remunerados, etc)')]


In [11]:
pprint(stores_handler.project_inputs.similarity_search("desenho", k=3))

[ProjectInput(subcategory=1, title='PRECISO DE DESIGNER GRFICO - #26', description='Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321', abilities=['Criação de logotipo', 'Ilustração']),
 ProjectInput(subcategory=1, title='PRECISO DE DESIGNER GRFICO - #21', description='Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321', abilities=['Criação de logotipo', 'Ilustração']),
 ProjectInput(subcategory=1, title='PRECISO DE DESIGNER GRFICO - #46', description='Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321', abilities=['Criação de logotipo', 'Ilustração'])]


In [12]:
pprint(stores_handler.project_predictions.similarity_search("escrita", k=3))

[ProjectPrediction(approve=True, subcategory=3, title='Escrever artigos para blog - #17', description='Preciso de redator para criação de conteúdo para meu blog https://meublog.com', reprovation_reason=None, reprovation_comment=None),
 ProjectPrediction(approve=True, subcategory=3, title='Escrever artigos para blog - #27', description='Preciso de redator para criação de conteúdo para meu blog https://meublog.com', reprovation_reason=None, reprovation_comment=None),
 ProjectPrediction(approve=True, subcategory=3, title='Escrever artigos para blog - #92', description='Preciso de redator para criação de conteúdo para meu blog https://meublog.com', reprovation_reason=None, reprovation_comment=None)]


In [13]:
stores_handler.subcategories.similarity_search("design")[0]

Subcategory(id=1, name='Design gráfico')

In [14]:
stores_handler.reprovation_reasons.similarity_search("duplicado")[0]

ReprovationReason(id=2, name='Projeto Duplicado', description='Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo')

In [15]:
stores_handler.project_inputs.similarity_search("desenho")[0]

ProjectInput(subcategory=1, title='PRECISO DE DESIGNER GRFICO - #26', description='Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321', abilities=['Criação de logotipo', 'Ilustração'])

In [16]:
stores_handler.project_predictions.similarity_search("escrita")[0]

ProjectPrediction(approve=True, subcategory=3, title='Escrever artigos para blog - #17', description='Preciso de redator para criação de conteúdo para meu blog https://meublog.com', reprovation_reason=None, reprovation_comment=None)

In [17]:
stores_handler.subcategories.get_by_id(1)

Subcategory(id=1, name='Design gráfico')

In [18]:
stores_handler.reprovation_reasons.get_by_id(2)


ReprovationReason(id=2, name='Projeto Duplicado', description='Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo')

In [19]:
stores_handler.project_inputs.get_by_id(3)

ProjectInput(subcategory=0, title='TRADUÇÃO DE TEXTO - #3', description='Preciso de tradutor para traduzir texto de inglês para português. Primeiramente será feito um teste não pago para depois ser decidido o freelancer que fará o projeto.', abilities=None)

In [20]:
stores_handler.project_predictions.get_by_id(4)

ProjectPrediction(approve=True, subcategory=6, title='Assistente virtual para tarefa pontual - #4', description='Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.', reprovation_reason=None, reprovation_comment=None)

## Create Selector and Examples Prompt

In [22]:
# from langchain.prompts import FewShotPromptTemplate, PromptTemplate
# from langchain.prompts.example_selector import (
#     MaxMarginalRelevanceExampleSelector,
#     SemanticSimilarityExampleSelector,
# )
# from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings

# example_prompt = PromptTemplate(
#     input_variables=["input", "output"],
#     template="Input: {input}\nOutput: {output}",
# )


In [23]:
# example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
#     # The list of examples available to select from.
#     examples,
#     # The embedding class used to produce embeddings which are used to measure semantic similarity.
#     embeddings=OpenAIEmbeddings(),
#     # The VectorStore class that is used to store the embeddings and do a similarity search over.
#     vectorstore_cls=FAISS,
#     # The number of examples to produce.
#     k=2,
# )
# mmr_prompt = FewShotPromptTemplate(
#     # We provide an ExampleSelector instead of examples.
#     example_selector=example_selector,
#     example_prompt=example_prompt,
#     prefix="Give the antonym of every input",
#     suffix="Input: {adjective}\nOutput:",
#     input_variables=["adjective"],
# )

In [24]:
# # Input is a feeling, so should select the happy/sad example as the first one
# print(mmr_prompt.format(adjective="worried"))